In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import numpy as np
import argparse
import time
from collections import defaultdict
from scipy import stats
import cv2
import os
from google.colab.patches import cv2_imshow

ModuleNotFoundError: No module named 'cv2'

A post that might help: https://stackoverflow.com/questions/1357403/how-to-cartoon-ify-an-image-programmatically

Blenderpy on github: https://github.com/TylerGubala/blenderpy
Install on linux issue: https://github.com/TylerGubala/blenderpy/issues/1

## Cartoonization-1
Cartoonization Method taken from https://github.com/lutianming/cartoonizer

In [2]:
def cartoonize(image):
    """
    convert image into cartoon-like image
    image: input PIL image
    """

    output = np.array(image)
    x, y, c = output.shape
    # hists = []
    for i in range(c):
        output[:, :, i] = cv2.bilateralFilter(output[:, :, i], 5, 50, 50)
        # hist, _ = np.histogram(output[:, :, i], bins=np.arange(256+1))
        # hists.append(hist)
    edge = cv2.Canny(output, 100, 200)

    output = cv2.cvtColor(output, cv2.COLOR_RGB2HSV)

    hists = []
    #H
    hist, _ = np.histogram(output[:, :, 0], bins=np.arange(180+1))
    hists.append(hist)
    #S
    hist, _ = np.histogram(output[:, :, 1], bins=np.arange(256+1))
    hists.append(hist)
    #V
    hist, _ = np.histogram(output[:, :, 2], bins=np.arange(256+1))
    hists.append(hist)

    C = []
    for h in hists:
        C.append(k_histogram(h))
    
    if(verbose):
      print("centroids: {0}".format(C))

    output = output.reshape((-1, c))
    for i in range(c):
        channel = output[:, i]
        index = np.argmin(np.abs(channel[:, np.newaxis] - C[i]), axis=1)
        output[:, i] = C[i][index]
    output = output.reshape((x, y, c))
    output = cv2.cvtColor(output, cv2.COLOR_HSV2RGB)

    contours, _ = cv2.findContours(edge,
                                   cv2.RETR_EXTERNAL,
                                   cv2.CHAIN_APPROX_NONE)
    # for i in range(len(contours)):
    #     tmp = contours[i]
    #     contours[i] = cv2.approxPolyDP(tmp, 2, False)
    cv2.drawContours(output, contours, -1, 0, thickness=1)
    return output

In [0]:
def update_C(C, hist):
    """
    update centroids until they don't change
    """
    while True:
        groups = defaultdict(list)
        #assign pixel values
        for i in range(len(hist)):
            if hist[i] == 0:
                continue
            d = np.abs(C-i)
            index = np.argmin(d)
            groups[index].append(i)

        new_C = np.array(C)
        for i, indice in groups.items():
            if np.sum(hist[indice]) == 0:
                continue
            new_C[i] = int(np.sum(indice*hist[indice])/np.sum(hist[indice]))
        if np.sum(new_C-C) == 0:
            break
        C = new_C
    return C, groups

In [0]:
def k_histogram(hist):
    """
    choose the best K for k-means and get the centroids
    """
    alpha = 0.001              # p-value threshold for normaltest
    N = 80                      # minimun group size for normaltest
    C = np.array([128])

    while True:
        C, groups = update_C(C, hist)

        #start increase K if possible
        new_C = set()     # use set to avoid same value when seperating centroid
        for i, indice in groups.items():
            #if there are not enough values in the group, do not seperate
            if len(indice) < N:
                new_C.add(C[i])
                continue

            # judge whether we should seperate the centroid
            # by testing if the values of the group is under a
            # normal distribution
            z, pval = stats.normaltest(hist[indice])
            if pval < alpha:
                #not a normal dist, seperate
                left = 0 if i == 0 else C[i-1]
                right = len(hist)-1 if i == len(C)-1 else C[i+1]
                delta = right-left
                if delta >= 3:
                    c1 = (C[i]+left)/2
                    c2 = (C[i]+right)/2
                    new_C.add(c1)
                    new_C.add(c2)
                else:
                    # though it is not a normal dist, we have no
                    # extra space to seperate
                    new_C.add(C[i])
            else:
                # normal dist, no need to seperate
                new_C.add(C[i])
        if len(new_C) == len(C):
            break
        else:
            C = np.array(sorted(new_C))
    return C

## Cartoonization-2

In [0]:
def make_image(a,N,p,input_image):

    print (a)
    print (N)
    print (p)
    #Phase 1 : color staircasing #########################################################################################################

    # Then We Do Bilateral Filtering with k1 = kernal size and N = number of iterations
    # We do Median Filtering
    # Color quantization floor factor = a

    for x in range(0,N):
        bilateral_filtimg = cv2.bilateralFilter(input_image,9,75,75)

    median_filtimg = cv2.medianBlur(bilateral_filtimg,5)

    [rows,cols,c] = median_filtimg.shape
    colorquantimg = median_filtimg
    for i in range(0,rows):
        for j in range(0,cols):
            xb = median_filtimg.item(i,j,0)
            xg = median_filtimg.item(i,j,1)
            xr = median_filtimg.item(i,j,2)  
            xb = math.floor(xb/a)*a 
            xg = math.floor(xg/a)*a
            xr = math.floor(xr/a)*a
            colorquantimg.itemset((i,j,0),xb)
            colorquantimg.itemset((i,j,1),xg)
            colorquantimg.itemset((i,j,2),xr)

        # Phase2 : Edge Extraction ############################################################################################################

        # Appy Median Filter to the image 
        # Canny Edge Detection
        # Dialation of the detected edges
        # Edgefilter
        #p = cv2.getTrackbarPos('Canny Threshold','Toonified Image')

    median_filtimg2 = cv2.medianBlur(input_image,5)
#     cv2_imshow(median_filtimg2)

    edges = cv2.Canny(median_filtimg2,p,2*p)
    dialateimg =  cv2.dilate(edges,np.ones((3,3),'uint8'))
    # dilateimg = edges
    edges_inv = cv2.bitwise_not(dialateimg)

#     cv2_imshow(edges)
#     cv2_imshow(edges_inv)

    # kernel = np.ones((5,5),np.uint8)
    # erosion_edges = cv2.erode(edges_inv,kernel,iterations = 1)
    # cv2_imshow(erosion_edges)

    ret,thresh = cv2.threshold(edges_inv,127,255,0)
    #cv2_imshow(thresh)
    contours, hierarchy = cv2.findContours(thresh,1,2)
    #(image, contours, contourIdx, color[, thickness[, lineType[, hierarchy[, maxLevel[, offset]]]]])
    img_contours = cv2.drawContours(image=thresh, contours=contours, contourIdx=-1, color=(0,0,0), thickness=0)
    print (img_contours)
    cv2_imshow(img_contours)
        #cv2.imshow('counters',img_contours)

        ############################### Recombine both the images ##############################################################################
    # global finalimg
    finalimg = colorquantimg
    for i in range(0,rows):
        for j in range(0,cols):
            if edges_inv.item(i,j) == 0:
                finalimg.itemset((i,j,0),0)
                finalimg.itemset((i,j,1),0)
                finalimg.itemset((i,j,2),0)
    # cv2_imshow(finalimg)       
    cv2.waitKey(0)  

    return finalimg

## Test Cartoonize-1

In [0]:
input = '/content/drive/My Drive/*.jpg'
image = cv2.imread(input)
# cv2_imshow(image)
start_time = time.time()
output = cartoonize(image)
end_time = time.time()
t = end_time-start_time
print('time: {0}s'.format(t))
# cv2_imshow(output)

NameError: ignored

## Test Cartoonize-2

In [0]:
def c2_driver(filename,a_ = 30, N_ = 5, p_ =55):
  a = a_ #from_=10, to=50
  N = N_ #from_=1, to=10
  p = p_ #from_=10, to=100

  input_image = cv2.imread(filename)
  final = make_image(a,N,p,input_image)
#   cv2_imshow(final) 

## Video Processing

Another movie library: https://pypi.org/project/mhmovie/

In [0]:
import moviepy.editor as mpe

def convert_video_cv2(input_file_location='/content/drive/My Drive/video.mp4', output_file_name = 'toonified.mp4', v=False):
  global verbose
  verbose = v
  video = mpe.VideoFileClip(input_file_location)
  audio = video.audio
  # Required only if you want to add background audio on the present audio
  # audio_background = mpe.AudioFileClip('some_background.mp3')
  # final_audio = mpe.CompositeAudioClip([audio, audio_background])

  cap = cv2.VideoCapture(input_file_location)

  if (cap.isOpened()== False):
      print("Error opening video stream or file")

  frame_width = int(cap.get(3))
  frame_height = int(cap.get(4))

  (major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

  if int(major_ver)  < 3 :
    fps = cap.get(cv2.cv.CV_CAP_PROP_FPS)
  else:
    fps = cap.get(cv2.CAP_PROP_FPS)
  
  encod = cv2.VideoWriter_fourcc('M','J','P','G')

  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  op_file = os.path.dirname(input_file_location) +'/'+ output_file_name
  out = cv2.VideoWriter(op_file,fourcc, fps, (frame_width,frame_height))
  
  i=0
  while(cap.isOpened()):
    ret, frame = cap.read()

    if ret == True:
      if (verbose):
        print("Frame Number: ",i)
      frame_ = cartoonize(frame)
      out.write(frame_)
      #cv2_imshow(frame)
      # Press Q on keyboard to  exit
      i=i+1
      # if cv2.waitKey(25) & 0xFF == ord('q'):
      #   break
    else:
      break
  cap.release()


  # We add the audio after the video has been made
  final_vid = mpe.VideoFileClip(op_file)
  final_clip = final_vid.set_audio(audio)
  final_clip.write_videofile(op_file)

In [0]:
def add_audio(input_file_location='/content/drive/My Drive/video.mp4', output_file_name = 'toonified.mp4'):
  video = mpe.VideoFileClip(input_file_location)
  audio = video.audio
  audio.write_audiofile(os.path.dirname(input_file_location) +'/'+'.mp3') 
  op_file = os.path.dirname(input_file_location) +'/'+ output_file_name
  out_vid = mpe.VideoFileClip(op_file)

In [0]:
import moviepy.editor as mpe

def convert_video_mpe(input_file_location='/content/drive/My Drive/video.mp4', output_file_name = 'toonified.mp4', v=False):
  global verbose
  verbose = v
  op_file = os.path.dirname(input_file_location) +'/'+ output_file_name
  video = mpe.VideoFileClip(input_file_location)
  audio = video.audio
  new_vid = video.fl_image( cartoonize )
  new_vid.write_videofile(op_file)

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)737280/45929032 bytes (1.6%)2039808/45929032 bytes (4.4%)3948544/45929032 bytes (8.6%)6651904/45929032 bytes (14.5%)10256384/45929032 bytes (22.3%)13926400/45929032 bytes (30.3%)17702912/45929032 bytes (38.5%)21397504/45929032 bytes (46.6%)25116672/45929032 bytes (54.7%)28516352/45929032 bytes (62.1%)32268288/45929032 bytes (70.3%)35913728/45929032 bytes (78.2%)3961651

## Caller function

In [0]:
convert_video_cv2('/content/drive/My Drive/<to location>/Agent327.mp4','Agent327_toonified_cv.mp4')
# add_audio('/content/drive/My Drive/<to location>/OMTM.mp4','OMTM_toonified.mp4')

In [0]:
# convert_video_mpe('/content/drive/My Drive/<to location>/numb.mp4','numb_toon.mp4')
convert_video_mpe('./numb.mp4','./numb_toon2.mp4')

[MoviePy] >>>> Building video /content/drive/My Drive/Creatives/numb_toon.mp4
[MoviePy] Writing audio in numb_toonTEMP_MPY_wvf_snd.mp3


100%|██████████| 4106/4106 [00:05<00:00, 697.62it/s]

[MoviePy] Done.
[MoviePy] Writing video /content/drive/My Drive/Creatives/numb_toon.mp4



100%|██████████| 4469/4469 [11:54<00:00,  6.26it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/drive/My Drive/Creatives/numb_toon.mp4 



In [0]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('inp', help='Input video file full path')
    parser.add_argument('output', help='Output file name. Only .mp4 format supported.')
    # parser.add_argument('video format', help='format of video to be saved')

    args = parser.parse_args()

    # image = Image.open(args.input)
    # image = cv2.imread(args.input)
    start_time = time.time()
    # output = cartoonize(image)
    convert_video(args.inp,args.output)
    end_time = time.time()
    t = end_time-start_time
    print('time: {0}s'.format(t))
    # cv2.imwrite(args.output, output)